# 5M строк датасет обработка

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

from geopy.geocoders import Nominatim

**Удаление выбросов**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Хакатоны/Сириус ИИ весна 2024/data5m.csv')
data = data.drop(['date', 'time'], axis=1)
data

,price,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type
0,6050000,59.805808,30.376141,2661,1,8,10,3,82.6,10.8,1
1,8650000,55.683807,37.297405,81,3,5,24,2,69.1,12.0,1
2,4000000,56.295250,44.061637,2871,1,5,9,3,66.0,10.0,1
3,1850000,44.996132,39.074783,2843,4,12,16,2,38.0,5.0,11
4,5450000,55.918767,37.984642,81,3,13,14,2,60.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5477001,19739760,55.804736,37.750898,3,1,8,17,4,93.2,13.8,11
5477002,12503160,55.841415,37.489624,3,2,17,32,2,45.9,6.6,11
5477003,8800000,56.283909,44.075408,2871,2,4,17,3,86.5,11.8,1
5477004,11831910,55.804736,37.750898,3,1,8,33,2,52.1,18.9,11


In [ ]:
new = data.copy() #дополнительно датасет анализировался аналогично исследованию, в другом файле



q_low = new['price'].quantile(0.25)
q_hi  = new['price'].quantile(0.75)

iqr = q_hi - q_low

new = new[(new['price'] >= q_low - iqr * 1.5) & (new['price'] <= q_hi + iqr * 1.5)]

new

,price,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type
0,6050000,59.805808,30.376141,2661,1,8,10,3,82.6,10.8,1
1,8650000,55.683807,37.297405,81,3,5,24,2,69.1,12.0,1
2,4000000,56.295250,44.061637,2871,1,5,9,3,66.0,10.0,1
3,1850000,44.996132,39.074783,2843,4,12,16,2,38.0,5.0,11
4,5450000,55.918767,37.984642,81,3,13,14,2,60.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...
5476996,6400000,55.904292,37.984368,81,3,4,17,3,82.0,10.6,1
5476997,7200000,59.772947,30.056530,3446,2,2,3,2,59.0,22.3,11
5476998,4900000,59.850103,30.357299,2661,1,2,5,1,31.0,6.0,1
5477000,9000000,44.051357,42.867573,2900,3,4,5,4,178.0,20.0,1


**Разделение на выборки**

In [ ]:
X, y = new.drop('price', axis=1), new['price'] #разделение на выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

**Обучение**

In [ ]:
model = CatBoostRegressor(iterations=500, #обучение
                          learning_rate=0.15,
                          depth=16,
                          loss_function='RMSE',
                          eval_metric='MAE',
                          verbose=10)

In [ ]:
model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

0:	learn: 1311909.7949290	test: 1309830.2379739	best: 1309830.2379739 (0)	total: 3.11s	remaining: 25m 53s
10:	learn: 673630.6976071	test: 672915.2049974	best: 672915.2049974 (10)	total: 34.6s	remaining: 25m 36s
20:	learn: 575305.4402466	test: 575886.0089887	best: 575886.0089887 (20)	total: 1m 7s	remaining: 25m 37s
30:	learn: 546274.6526404	test: 547461.2258303	best: 547461.2258303 (30)	total: 1m 38s	remaining: 24m 44s
40:	learn: 527584.1113866	test: 529265.5327074	best: 529265.5327074 (40)	total: 2m 9s	remaining: 24m 13s
50:	learn: 514667.1004505	test: 516634.5036922	best: 516634.5036922 (50)	total: 2m 41s	remaining: 23m 38s
60:	learn: 505326.2211224	test: 507668.4352772	best: 507668.4352772 (60)	total: 3m 11s	remaining: 22m 55s
70:	learn: 496759.8530092	test: 499386.7222735	best: 499386.7222735 (70)	total: 3m 41s	remaining: 22m 16s
80:	learn: 489131.9967031	test: 492259.4877453	best: 492259.4877453 (80)	total: 4m 12s	remaining: 21m 45s
90:	learn: 482737.2019536	test: 486160.3426171	be

**Оценка**

In [ ]:
predictions = model.predict(X_test)

print(mae(y_test, predictions))

413316.2372435786


In [ ]:
model.save_model("catboost_5m_3.cbm")